# Lesson 17 - Design Patterns

In [21]:
#include <iostream>
#include <string>
#include <vector>
#include <fstream>
#include <memory>

## Parameters packs

In [ ]:
template<typename ...Ts>
void MyPPFunction(Ts... args) {
    std::cout << "I was passed " << sizeof...(args) << " arguments" << std::endl;
}

In [ ]:
MyPPFunction(1, 3, 4);
MyPPFunction("Hello world", "Hi there");

In [ ]:
template<typename ...Ts>
std::vector<std::common_type_t<Ts...>> make_vector(Ts... args) {
    return std::vector<std::common_type_t<Ts...>> {args...};
}

In [ ]:
auto my_vector {make_vector(1, 2, 3, 4, 5, 6)};
std::cout << "my_vector has " << my_vector.size() << " elements" << std::endl;

In [ ]:
template<typename T>
void PrintList(T arg) {
    std::cout << arg << std::endl;
}

template<typename T, typename ...Ts>
void PrintList(T arg, Ts... args) {
    std::cout << arg << ", ";
    PrintList(args...);
}

In [ ]:
PrintList(1, 2, 3, "hello world", 5.5, 7, "end");

## Initialiser lists

In [2]:
template<typename T>
void MyInitFunction(std::initializer_list<T> args) {
    for (const auto& arg: args) {
        std::cout << "Argument: " << arg << std::endl;
    }
}

In [3]:
MyInitFunction({1, 2, 3, 4, 5});
MyInitFunction({"Hi", "there"});

Argument: 1
Argument: 2
Argument: 3
Argument: 4
Argument: 5
Argument: Hi
Argument: there


In [4]:
MyInitFunction({1, 2, 3, 4, "Hi"}); // Error - not the same types

input_line_10:2:2: error: no matching function for call to 'MyFunction'
 MyFunction({1, 2, 3, 4, "Hi"}); // Error - not the same types
 ^~~~~~~~~~
input_line_8:2:6: note: candidate template ignored: deduced conflicting types for parameter 'T' ('int' vs. 'const char *')
void MyFunction(std::initializer_list<T> args) {
     ^


Interpreter Error: 

In [5]:
// Note - no template now
void MyInitIntFunction(std::initializer_list<int> args) {
    for (const auto& arg: args) {
        std::cout << "Argument: " << arg << std::endl;
    }
}

In [6]:
MyInitIntFunction({1, 2, 3, 4});

Argument: 1
Argument: 2
Argument: 3
Argument: 4


In [7]:
MyInitIntFunction({1.2, 1.3}); // Error - not int

input_line_13:2:21: error: type 'double' cannot be narrowed to 'int' in initializer list [-Wc++11-narrowing]
 MyInitIntFunction({1.2, 1.3}); // Error - not int
                    ^~~
input_line_13:2:21: note: insert an explicit cast to silence this issue
 MyInitIntFunction({1.2, 1.3}); // Error - not int
                    ^~~
                    static_cast<int>( )
input_line_13:2:26: error: type 'double' cannot be narrowed to 'int' in initializer list [-Wc++11-narrowing]
 MyInitIntFunction({1.2, 1.3}); // Error - not int
                         ^~~
input_line_13:2:26: note: insert an explicit cast to silence this issue
 MyInitIntFunction({1.2, 1.3}); // Error - not int
                         ^~~
                         static_cast<int>( )
input_line_13:2:21: warning: implicit conversion from 'double' to 'int' changes value from 1.2 to 1 [-Wliteral-conversion]
 MyInitIntFunction({1.2, 1.3}); // Error - not int
                   ~^~~
input_line_13:2:26: warning: implicit convers

Interpreter Error: 

## Design patterns

This is a huge area in software development - there are [a lot of books on this topic](https://www.amazon.co.uk/s?k=design+patterns&ref=nb_sb_noss).

### Singleton

In [10]:
struct Configuration {
    std::unique_ptr<std::ifstream> input_file;
    // Any other program-wide configuration options
};

In [22]:
class ProgramConfiguration {
public:
    Configuration& GetConfiguration() {return configuration_;}
    static ProgramConfiguration& GetInstance() {
        static ProgramConfiguration instance; // Create an instance
        return instance; // Return the only instance
    }
private:
    ProgramConfiguration(); // Default constructor is private
    ProgramConfiguration(const ProgramConfiguration& other) = delete; // Do not allow the instance to be copied
    ProgramConfiguration(const ProgramConfiguration&& other) = delete; // Do not allow the instance to be moved
    ProgramConfiguration& operator=(const ProgramConfiguration &rhs) const = delete; //disallow copy-assignment operator
    ProgramConfiguration& operator=(const ProgramConfiguration &&rhs) const = delete; //disallow move-assignment operator
    ~ProgramConfiguration() {} // Private destructor controls who can destroy the object
    Configuration configuration_;
};

input_line_28:1:7: error: redefinition of 'ProgramConfiguration'
class ProgramConfiguration {
      ^
input_line_17:1:7: note: previous definition is here
class ProgramConfiguration {
      ^


Interpreter Error: 

In [23]:
ProgramConfiguration::ProgramConfiguration() : configuration_ {} {
    std::string filename {"_config.yml"}; // Load this from a configuration file, for example.
    configuration_.input_file = std::make_unique<std::ifstream>(filename);
}

input_line_29:1:66: error: expected '{' or ','
ProgramConfiguration::ProgramConfiguration() : configuration_ {} void __cling_Un1Qu315(void* vpClingValue) {
                                                                 ^
In file included from input_line_5:1:
In file included from /srv/conda/envs/notebook/include/xeus/xinterpreter.hpp:17:
In file included from /srv/conda/envs/notebook/include/xeus/xcomm.hpp:19:
In file included from /srv/conda/envs/notebook/include/nlohmann/json.hpp:45:
In file included from /srv/conda/envs/notebook/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/memory:80:
/srv/conda/envs/notebook/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/bits/unique_ptr.h:825:34: error: no matching constructor for initialization of 'std::basic_ifstream<char>'
    { return unique_ptr<_Tp>(new _Tp(std::forward<_Args>(__args)...)); }
                              

Interpreter Error: 

### Factory